# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse               Shipments  \
0  Jun 12 2023  2:01PM  264600        10                 9566351   
1  Jun 12 2023  2:01PM  264600        10                 9566352   
2  Jun 12 2023  1:59PM  264599        10         SNAP20230608-1E   
3  Jun 12 2023  1:55PM  264598        10  SNAP612231-Hold_Rx(MI)   
4  Jun 12 2023  1:45PM  264597        10              Eme-113179   
5  Jun 12 2023  1:45PM  264597        10             Enova-11943   
6  Jun 12 2023  1:35PM  264595        10                   38532   
7  Jun 12 2023  1:35PM  264594        10               MSP222619   
8  Jun 12 2023  1:35PM  264594        10               MSP222620   
9  Jun 12 2023  1:35PM  264594        10               MSP222621   

                       Customer ShipmentStatus  
0  Snap Medical Industries, LLC       Released  
1  Snap Medical Industries, LLC       Released  
2  Snap Medical Industries, LLC       Released  
3  Snap Medical Industries, LLC       Released  
4                   Emerginnova       Released  
5                   Emerginnova       Released  
6           Digital Brands, LLC       Released  
7              Methapharm, Inc.       Released  
8              Methapharm, Inc.       Released  
9              Methapharm, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
21  264595       Released          1
22  264597       Released          2
23  264598       Released          1
24  264599       Released          1
25  264600       Released          2

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
264595                NaN        NaN       1.0
264597                NaN        NaN       2.0
264598                NaN        NaN       1.0
264599                NaN        NaN       1.0
264600                NaN        NaN       2.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
264535                0.0        0.0       1.0
264537                0.0        1.0       0.0
264547                8.0       18.0       5.0
264555                0.0        0.0       1.0
264558                0.0       21.0      19.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
264535                  0          0         1
264537                  0          1         0
264547                  8         18         5
264555                  0          0         1
264558                  0         21        19

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               264535          0          0         1
1               264537          0          1         0
2               264547          8         18         5
3               264555          0          0         1
4               264558          0         21        19

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               264535                            1
1               264537                   1         
2               264547         8        18        5
3               264555                            1
4               264558                  21       19

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                      Customer
0   Jun 12 2023  2:01PM  264600        10  Snap Medical Industries, LLC
2   Jun 12 2023  1:59PM  264599        10  Snap Medical Industries, LLC
3   Jun 12 2023  1:55PM  264598        10  Snap Medical Industries, LLC
4   Jun 12 2023  1:45PM  264597        10                   Emerginnova
6   Jun 12 2023  1:35PM  264595        10           Digital Brands, LLC
7   Jun 12 2023  1:35PM  264594        10              Methapharm, Inc.
14  Jun 12 2023  1:34PM  264593        10                     Hush Hush
17  Jun 12 2023  1:34PM  264591        10              Methapharm, Inc.
18  Jun 12 2023  1:25PM  264588        19           AdvaGen Pharma, Ltd
19  Jun 12 2023 12:56PM  264585        10           Alliance Pharma LTD

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                      Customer  \
0  Jun 12 2023  2:01PM  264600        10  Snap Medical Industries, LLC   
1  Jun 12 2023  1:59PM  264599        10  Snap Medical Industries, LLC   
2  Jun 12 2023  1:55PM  264598        10  Snap Medical Industries, LLC   
3  Jun 12 2023  1:45PM  264597        10                   Emerginnova   
4  Jun 12 2023  1:35PM  264595        10           Digital Brands, LLC   
5  Jun 12 2023  1:35PM  264594        10              Methapharm, Inc.   
6  Jun 12 2023  1:34PM  264593        10                     Hush Hush   
7  Jun 12 2023  1:34PM  264591        10              Methapharm, Inc.   
8  Jun 12 2023  1:25PM  264588        19           AdvaGen Pharma, Ltd   
9  Jun 12 2023 12:56PM  264585        10           Alliance Pharma LTD   

  Completed Executing Released  
0                            2  
1                            1  
2                            1  
3                            2  
4                            1  
5                            7  
6                            3  
7                            1  
8                            1  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                      Customer  \
0  Jun 12 2023  2:01PM  264600        10  Snap Medical Industries, LLC   
1  Jun 12 2023  1:59PM  264599        10  Snap Medical Industries, LLC   
2  Jun 12 2023  1:55PM  264598        10  Snap Medical Industries, LLC   
3  Jun 12 2023  1:45PM  264597        10                   Emerginnova   
4  Jun 12 2023  1:35PM  264595        10           Digital Brands, LLC   
5  Jun 12 2023  1:35PM  264594        10              Methapharm, Inc.   
6  Jun 12 2023  1:34PM  264593        10                     Hush Hush   
7  Jun 12 2023  1:34PM  264591        10              Methapharm, Inc.   
8  Jun 12 2023  1:25PM  264588        19           AdvaGen Pharma, Ltd   
9  Jun 12 2023 12:56PM  264585        10           Alliance Pharma LTD   

  Released Executing Completed  
0        2                      
1        1                      
2        1                      
3        2                      
4        1                      
5        7                      
6        3                      
7        1                      
8        1                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Jun 12 2023  2:01PM  264600        10  Snap Medical Industries, LLC   
1  Jun 12 2023  1:59PM  264599        10  Snap Medical Industries, LLC   
2  Jun 12 2023  1:55PM  264598        10  Snap Medical Industries, LLC   
3  Jun 12 2023  1:45PM  264597        10                   Emerginnova   
4  Jun 12 2023  1:35PM  264595        10           Digital Brands, LLC   

  Released Executing Completed  
0        2                      
1        1                      
2        1                      
3        2                      
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Jun 12 2023  2:01PM  264600        10  Snap Medical Industries, LLC   
1  Jun 12 2023  1:59PM  264599        10  Snap Medical Industries, LLC   
2  Jun 12 2023  1:55PM  264598        10  Snap Medical Industries, LLC   
3  Jun 12 2023  1:45PM  264597        10                   Emerginnova   
4  Jun 12 2023  1:35PM  264595        10           Digital Brands, LLC   

   Released  Executing  Completed  
0       2.0        NaN        NaN  
1       1.0        NaN        NaN  
2       1.0        NaN        NaN  
3       2.0        NaN        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Jun 12 2023  2:01PM  264600        10  Snap Medical Industries, LLC   
1  Jun 12 2023  1:59PM  264599        10  Snap Medical Industries, LLC   
2  Jun 12 2023  1:55PM  264598        10  Snap Medical Industries, LLC   
3  Jun 12 2023  1:45PM  264597        10                   Emerginnova   
4  Jun 12 2023  1:35PM  264595        10           Digital Brands, LLC   

   Released  Executing  Completed  
0       2.0        0.0        0.0  
1       1.0        0.0        0.0  
2       1.0        0.0        0.0  
3       2.0        0.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3704182      34.0        5.0        0.0
15          529105      24.0       39.0        8.0
18          264535       1.0        0.0        0.0
19          793711       3.0        0.0        0.0
20          529163      22.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3704182      34.0        5.0        0.0
1        15   529105      24.0       39.0        8.0
2        18   264535       1.0        0.0        0.0
3        19   793711       3.0        0.0        0.0
4        20   529163      22.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      34.0        5.0        0.0
1        15      24.0       39.0        8.0
2        18       1.0        0.0        0.0
3        19       3.0        0.0        0.0
4        20      22.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   34.0
1        15  Released   24.0
2        18  Released    1.0
3        19  Released    3.0
4        20  Released   22.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   18   19    20
Status                               
Completed   0.0   8.0  0.0  0.0   0.0
Executing   5.0  39.0  0.0  0.0   0.0
Released   34.0  24.0  1.0  3.0  22.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   18   19    20
0          Completed   0.0   8.0  0.0  0.0   0.0
1          Executing   5.0  39.0  0.0  0.0   0.0
2           Released  34.0  24.0  1.0  3.0  22.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   18   19    20
0  Completed   0.0   8.0  0.0  0.0   0.0
1  Executing   5.0  39.0  0.0  0.0   0.0
2   Released  34.0  24.0  1.0  3.0  22.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()